In [2]:
import requests
import json
import forecastio
from geopy.geocoders import Nominatim
import pandas as pd
import import_ipynb
from Austin_airbnb import Austin_airbnb
import datetime as dt
from config import api_key

In [3]:
geolocator = Nominatim(user_agent="ETL_Project")

location = geolocator.geocode("Austin, TX")

ATX_coord = []
ATX_coord.append(location.latitude)
ATX_coord.append(location.longitude)

ATX_coord

[30.2711286, -97.7436995]

In [4]:
forecast = forecastio.load_forecast(api_key, ATX_coord[0], ATX_coord[1])

byDay = forecast.daily()
print(byDay.summary)
print(byDay.icon)
print(byDay.data[4].temperatureHigh)

Rain on Wednesday and Thursday, with high temperatures falling to 46°F next Saturday.
rain
68.26


In [5]:
temp_high = []
temp_low = []
precip_perc = []
precip_type = []
precip_int = []
date = []

for i in byDay.data:
    date.append(i.time.date())
    temp_high.append(i.temperatureHigh)
    temp_low.append(i.temperatureLow)
    precip_perc.append(i.precipProbability)
    precip_int.append(i.precipIntensity)
    try:
        precip_type.append(i.precipType)
    except:
        precip_type.append("none")

date

dates = [dt.datetime.strftime(i, '%Y-%m-%d') for i in date]
dates

['2018-12-22',
 '2018-12-23',
 '2018-12-24',
 '2018-12-25',
 '2018-12-26',
 '2018-12-27',
 '2018-12-28',
 '2018-12-29']

In [6]:
df = pd.DataFrame({"Date": dates, 
                     "High Temp": temp_high, 
                     "Low Temp": temp_low, 
                     "Precipitation Probability": precip_perc,
                     "Precipitation Intensity": precip_int,
                     "Precipitation Type": precip_type
                    })
df.dtypes

Date                          object
High Temp                    float64
Low Temp                     float64
Precipitation Probability    float64
Precipitation Intensity      float64
Precipitation Type            object
dtype: object

In [7]:
ATX_table = pd.merge(df, Austin_airbnb, on="Date")
ATX_table

,Date,High Temp,Low Temp,Precipitation Probability,Precipitation Intensity,Precipitation Type,Under $50,$50-100,$100-150,$150-200,$200-250
0,2018-12-22,73.59,45.85,0.04,0.0002,rain,532,1220,1039,678,360
1,2018-12-23,59.67,40.37,0.08,0.0001,rain,615,1335,980,610,294
2,2018-12-24,57.70,54.45,0.08,0.0007,rain,592,1291,963,602,273
3,2018-12-25,68.11,63.68,0.28,0.0035,rain,584,1267,960,598,270
4,2018-12-26,68.26,57.16,0.85,0.0329,rain,607,1286,996,579,276
5,2018-12-27,68.54,45.23,0.51,0.0101,rain,624,1274,955,588,276
6,2018-12-28,57.18,41.20,0.00,0.0000,none,525,1071,877,684,360
7,2018-12-29,45.24,36.65,0.36,0.0020,rain,523,1044,858,651,380
